In [8]:
import snappy
import sys
from snappy import (ProductIO, ProductUtils, ProgressMonitor)

In [9]:
jpy = snappy.jpy

# More Java type definitions required for image generation
Color = jpy.get_type('java.awt.Color')
ColorPoint = jpy.get_type('org.esa.snap.core.datamodel.ColorPaletteDef$Point')
ColorPaletteDef = jpy.get_type('org.esa.snap.core.datamodel.ColorPaletteDef')
ImageInfo = jpy.get_type('org.esa.snap.core.datamodel.ImageInfo')
ImageLegend = jpy.get_type('org.esa.snap.core.datamodel.ImageLegend')
ImageManager = jpy.get_type('org.esa.snap.core.image.ImageManager')
JAI = jpy.get_type('javax.media.jai.JAI')
RenderedImage = jpy.get_type('java.awt.image.RenderedImage')

# Disable JAI native MediaLib extensions 
System = jpy.get_type('java.lang.System')
System.setProperty('com.sun.media.jai.disableMediaLib', 'true')

'true'

In [10]:
def write_image(band, filename, format):
    im = ImageManager.getInstance().createColoredBandImage([band], band.getImageInfo(), 0)
    JAI.create("filestore", im, filename, format)

def write_rgb_image(bands, filename, format):
    image_info = ProductUtils.createImageInfo(bands, True, ProgressMonitor.NULL)
    im = ImageManager.getInstance().createColoredBandImage(bands, image_info, 0)
    JAI.create("filestore", im, filename, format)

In [11]:
file = '/eodata/Envisat/Meris/FRS/2011/09/25/MER_FRS_1PPEPA20110925_090423_000005093107_00036_50050_5197.N1'

product = ProductIO.readProduct(file)
band = product.getBand('radiance_13')
print(band)

org.esa.snap.core.datamodel.Band[radiance_13,uint16,4481,11585,12,864.87604,20.047,953.5812]


In [12]:
# The colour palette assigned to pixel values 0, 50, 100 in the band's geophysical units
points = [ColorPoint(0.0, Color.YELLOW), 
          ColorPoint(50.0, Color.RED), 
          ColorPoint(100.0, Color.BLUE)]
cpd = ColorPaletteDef(points)
ii = ImageInfo(cpd)
band.setImageInfo(ii)

image_format = 'PNG'
write_image(band, 'snappy_write_image.png', image_format)

RuntimeError: java.lang.NullPointerException

In [6]:
legend = ImageLegend(band.getImageInfo(), band)
legend.setHeaderText(band.getName())
legend_image = legend.createImage()

# This cast is need because otherwise jpy can't evaluate which method to call
# This is considered as an issue of jpy (https://github.com/bcdev/jpy/issues/89)
rendered_legend_image = jpy.cast(legend_image, RenderedImage)
JAI.create("filestore", rendered_legend_image, 'snappy_write_image_legend.png', image_format)

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


javax.media.jai.RenderedOp(objectRef=0x52586b8)

In [7]:
red = product.getBand('radiance_7')
green = product.getBand('radiance_5')
blue = product.getBand('radiance_3')

write_rgb_image([red, green, blue], 'snappy_write_image_rgb.png', image_format)

In [12]:
im = Image.open("snappy_write_image_rgb.png")
nx, ny = im.size
im2 = im.resize((int(nx*0.2), int(ny*0.2)), Image.BICUBIC)
im2.save("snappy_write_image_rgb_res.png")

In [13]:
layer=ImageLayer()
box=layer.getbb(file)
layer.addImageLayer('snappy_write_image_rgb_res.png', box, 'Meris_RGB')
layer.showLayer()

org.esa.snap.core.datamodel.Product[name=MER_FRS_1PPEPA20110925_090423_000005093107_00036_50050_5197.N1]
63.320927999999995
